In [ ]:
from pyspark import SparkContext
from config.config import PRODUCER_CONFIG
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json

KAFKA_TOPIC = PRODUCER_CONFIG['KAFKA_TOPIC']
BOOTSTRAP_SERVER = "localhost:9092"
sc = SparkContext.getOrCreate()

In [ ]:
ssc = StreamingContext(sc,2) #stream each one second
ssc.checkpoint("./checkpoint")
data = KafkaUtils.createDirectStream(ssc, [KAFKA_TOPIC],
                                  {"metadata.broker.list": BOOTSTRAP_SERVER})
def updateFunc(newVal, oldVal):  
    if oldVal is None:
        oldVal=([], [], [])
    old_id, old_desc, old_publish = oldVal
    new_id = [x[0] for x in newVal]
    new_desc = [x[1] for x in newVal]
    new_publish = [x[2] for x in newVal]
    return (old_id+new_id, old_desc+new_desc, old_publish+new_publish)
def saveToParquet(rdd):
    if not rdd.isEmpty():
        df=rdd.toDF()
        df.write.format("parquet").mode('append').save(PRODUCER_CONFIG['HDFS_FOLDER'])
def prepare_rdd(data, window_length=2, sliding_interval=2):
    data = data.window(window_length, sliding_interval).map(lambda x:json.loads(x[1]))
    data = data.map(
         lambda x: (x['video_id'], x['description'], x['published_at'], x['channel_title'], x['video_title'])
    )
    data.foreachRDD(saveToParquet)
    return data
# run the function
result = prepare_rdd(data)
# Print
result.pprint()
ssc.start()
ssc.awaitTermination()